## Dependencies

In [127]:
from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle
from PIL import Image
import numpy as np
import os

os.chdir("../data")

## Reduce dataset (optional)

In [53]:
cats, dogs = 0, 0
max_images = 1000
counter = {'cat': cats, 'dog': dogs}

for filename in os.listdir("cat_dog"):
    animal, number = filename.split(".")[:2]
    if counter[animal] >= max_images:
        path = os.path.join("cat_dog", filename)
        os.remove(path)
    else:
        counter[animal] += 1

## Preprocessing

In [128]:
if not os.path.exists("preprocessed"):
    os.makedirs("preprocessed")

In [212]:
for file in os.listdir("cat_dog"):
    path = os.path.join("cat_dog", file)
    image = Image.open(path)
    image = image.resize((64, 64))
    image = image.convert('L')
    path = os.path.join("preprocessed", file)
    image.save(path)

## Convert to NumPy

In [201]:
features = []
labels = []

classes = ["cat", "dog"]
for file in os.listdir("preprocessed"):
    path = os.path.join("preprocessed", file)
    image = Image.open(path)
    features.append(np.array(image))
    label, number = file.split(".")[:2]
    classid = classes.index(label)
    labels.append([classid, 1-classid])

features = np.array(features)
labels = np.array(labels)


features, labels = shuffle(features, labels)
print(features.shape, labels.shape)

(2000, 32, 32, 3) (2000, 2)


## Train Test Split

In [202]:
X_train, X_test, y_train, y_test = train_test_split(features, labels, test_size=0.15, random_state=42)

## Train model

In [229]:
from keras.models import Sequential
from keras.layers import Dense, Conv2D, Flatten, MaxPooling2D, Dropout, Input

model = Sequential()
model.add(Input(shape=(64,64,1)))
model.add(Conv2D(32, kernel_size=3, activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Conv2D(64, kernel_size=3, activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.1))
model.add(Dense(128, activation='relu'))
model.add(Dense(2, activation='softmax'))

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [230]:
history = model.fit(X_train, y_train, epochs=30, batch_size=32, validation_data=(X_test, y_test), verbose=0)

In [232]:
accuracy = model.evaluate(X_test, y_test, verbose=0)[1]
print("Accuracy: ", round(accuracy * 100, 2))

Accuracy:  60.33


In [234]:
model.save("catdog.keras")